The structure based pharmacophore was developed using Openpharmacophore program (https://github.com/uibcdf/OpenPharmacophore) accessed on 22th April 2022

In [ ]:
from openpharmacophore import StructuredBasedPharmacophore
from IPython.display import Image
pharmacophore_1 = StructuredBasedPharmacophore().from_pdb("./Bp_receptor.pdb", 
                                                          radius=1.0, 
                                                          ligand_id="UNK:A:757", 
                                                          hydrophobics="plip")

#pharmacophore_1.remove_point(0)
pharmacophore_1._pharmacophoric_points

In [2]:
view = pharmacophore_1.show()
view

NGLWidget()

In [2]:
pharmacophore_1.remove_points([0,2,3])

In [3]:
pharmacophore_1.to_moe("./mypharmacophore.ph4")

In [4]:
view = pharmacophore_1.show()
view

NGLWidget()

In [3]:
pharmacophore_1.remove_points([0,2,3])
#pharmacophore_1.remove_points([1,2,4])

In [5]:
pharmacophore_1.to_pharmer("./abhishek_strphar.json") 

In [6]:
pharmacophore_1._pharmacophoric_points

[PharmacophoricPoint(feat_type=hb acceptor; center=(64.91, 10.11, 2.64); radius: 1.0; direction=(-0.9471, -0.1009, 0.3045)),
 PharmacophoricPoint(feat_type=hb donor; center=(64.67, 7.67, 3.83); radius: 1.0; direction=(0.7997, 0.5583, 0.2207)),
 PharmacophoricPoint(feat_type=hydrophobicity; center=(62.09, 11.17, 1.5); radius=1.0),
 PharmacophoricPoint(feat_type=hydrophobicity; center=(63.9, 4.27, 4.58); radius=1.0)]

In [8]:
def _get_rdkit_pharmacophore(pharmacophore):
        """ Transfor a pharmacophore to an rdkit pharmacophore
        
            Parameters
            ----------
            pharmacophore : openpharmacophore.Pharmacophore
                The pharmacophore.
                
            Returns
            -------
            rdkit.Chem.Pharm3D.Pharmacophore
                An rdkit pharmacophore with the radii of the points applied.
        """
        rdkit_pharmacophore, radii = pharmacophore.to_rdkit()
        apply_radii_to_bounds(radii, rdkit_pharmacophore)

        return rdkit_pharmacophore

In [13]:
from openpharmacophore import Pharmacophore, LigandBasedPharmacophore, StructuredBasedPharmacophore
from openpharmacophore.io.mol_suppliers import smiles_mol_generator, smi_has_header_and_id, mol2_mol_generator
from openpharmacophore.algorithms.alignment import apply_radii_to_bounds, transform_embeddings
from openpharmacophore._private_tools.exceptions import NoMatchesError, OpenPharmacophoreIOError, OpenPharmacophoreNotImplementedError, OpenPharmacophoreTypeError
from openpharmacophore._private_tools.screening_arguments import check_virtual_screening_kwargs, is_3d_pharmacophore
# Third party
import pandas as pd
from rdkit import RDConfig, Chem, RDLogger, DataStructs
from rdkit.Chem import ChemicalFeatures, rdDistGeom, Descriptors
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D
from rdkit.Chem.Pharm2D.Generate import Gen2DFingerprint
from rdkit.Chem.Pharm3D import EmbedLib
from tqdm.auto import tqdm
# Standard library
import bisect
from collections import namedtuple
import json
from multiprocessing import Pool, Queue, Manager, Value
from operator import itemgetter
import os
from typing import Callable, List, Tuple, TypeVar

matches=[]
Match = namedtuple("Match", ["score", "id", "mol"]) # Named tuple to store molecules that match a pharmacophore
def _align_molecule(mol, pharmacophore, matches, featFactory, sort=False):
        """ Align a molecule to a given pharmacophore.
        
            Uses rdkit alignment algorithm

            Parameters
            ----------
            mol : rdkit.Chem.Mol
                Molecule to align.
                
            matches : list
                If a moleculed is matched to the pharmacophore it will be appended to this list.
                
            pharmacophore : rdkit.Chem.Pharm3D.Pharmacophore
                An rdkit pharmacophore

            featFactory : rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeatureFactory
                The feature factory.
            
            sort : bool, default=False
                Whether to sort the list with the matches

        """
        #cids = AllChem.EmbedMultipleConfs(mol, numConfs=500, maxAttempts=1000,pruneRmsThresh=0.1)
        bounds_matrix = rdDistGeom.GetMoleculeBoundsMatrix(mol)
        # Check if the molecule features can match with the pharmacophore.
        can_match, all_matches = EmbedLib.MatchPharmacophoreToMol(mol, featFactory, pharmacophore)
        #print(can_match)
        #print(all_matches)
        # all_matches is a list of tuples where each tuple contains the chemical features
        if can_match:
            # Match the molecule to the pharmacophore without aligning it
            failed, bounds_matrix_matched, matched_mols, match_details = EmbedLib.MatchPharmacophore(all_matches, 
                                                                                            bounds_matrix,
                                                                                            pharmacophore, 
                                                                                            useDownsampling=False)
            #print(matched_mols)
            #print(failed, bounds_matrix_matched, matched_mols, match_details)
            n=0
            if failed:
                print('failed')
                return
            else:
                #print('not failed')
                n+=1
        else:
            return
        bounds = rdDistGeom.GetMoleculeBoundsMatrix (mol)
        pList = EmbedLib.GetAllPharmacophoreMatches (all_matches,bounds_matrix,pharmacophore)
        #print(len (pList))
        AtomIds_list = []

        '''for i in  range ( len (pList [ 2 ])):
            p = pList [2] [i]
        print (p.GetFamily (), ':' , p.GetAtomIds ())
        AtomIds_list.append (p.GetAtomIds ())'''
                                                                                            
        #print('Hello')
        atom_match = [list(x.GetAtomIds()) for x in matched_mols]
        #print(atom_match)
        #print('compounds_passed '+str(n))
        try: 
            mol_H = Chem.AddHs(mol)
        except:
            print('Ifailed')
        # Embed molecule onto the pharmacophore
        # embeddings is a list of molecules with a single conformer
        try:
           b_matrix, embeddings, num_fail = EmbedLib.EmbedPharmacophore(mol, atom_match, pharmacophore, count=10)
        except:
              return 
        #print(b_matrix, embeddings, num_fail)
         
        # Align embeddings to the pharmacophore 
        SSDs,confs = transform_embeddings(pharmacophore, embeddings, atom_match)
        #print(SSDs)
        if len(SSDs) == 0:
            n-=1
            return
        
        best_fit_index = max(enumerate(SSDs), key=itemgetter(1))[0]
        try:
            mol_id = mol.GetProp("_Name")
        except:
            mol_id = None

        matched_mol = Match(SSDs[best_fit_index], mol_id, embeddings[best_fit_index])
        #print(matched_mol)
        if sort:
            print('sort')
            # Append to list in ordered manner
            try:
                # Case when a molecule is repeated. It will throw an error since bisect
                # cannot compare molecules.
                bisect.insort(matches, matched_mol)
            except:
                print('not sort')
                return
        else:
            matches.append(matched_mol)
        return matches

In [14]:
from rdkit.Chem import AllChem
#pharmacophore_1.remove_point(1)
pharm=_get_rdkit_pharmacophore(pharmacophore_1)
_factory = ChemicalFeatures.BuildFeatureFactory(os.path.join(RDConfig.RDDataDir,
                                                                     'BaseFeatures.fdef'))
x=[]
#for mol in Chem.SDMolSupplier('./abhishek_missed/mm.sdf', removeHs=False):
for mol in Chem.SDMolSupplier('./abhishek_missed/A_min2.sdf', removeHs=False):
    x.append(_align_molecule(mol, pharm, matches, _factory, sort=False))
x

[[Match(score=0.09170688312218189, id='10a', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC8A33E00>),
  Match(score=0.09087338529139366, id='10b', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC8A33D60>),
  Match(score=0.09166871494041118, id='10c', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC8A33950>),
  Match(score=0.09219852877284573, id='10d', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC8A33A90>),
  Match(score=0.09202575005999059, id='10e', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC75314A0>),
  Match(score=0.09199848585928372, id='10f', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC72DAF90>),
  Match(score=0.08815389023047082, id='10g', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC7531590>),
  Match(score=0.08395943864004618, id='10k', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC8A33F40>),
  Match(score=0.09134956079674428, id='10l', mol=<rdkit.Chem.rdchem.Mol object at 0x0000020DC858CBD0>),
  Match(score=0.09173163845882737, id='10m', mol=<rdkit.Chem.rdc

In [15]:
ls=[]
for i in x[0]:
    ls.append(i.mol)
len(ls)

29

In [16]:
ls=[]
for i in x[0]:
    if i is not None:
       ls.append(i.mol)
len(ls)

29

In [17]:
len(x)

30

In [19]:
name='c4.sdf'
if os.path.exists(name):
    os.remove(name)
sdfile=open(name,'a')
for i in x[0]:
  w = Chem.SDWriter(sdfile)
  w.write(i.mol)
w.close()

In [20]:
sdfile.close()

In [1]:
x[0]

NameError: name 'x' is not defined

In [13]:
ls=[]
for i in x:
    if i is not None:
       ls.append(i)
len(ls)

29

In [47]:
from rdkit.Chem import Draw
for m in x:
    #print(m[0].numAtoms)
    AllChem.EmbedMolecule(m[0], AllChem.ETKDGv2())
    #writer = Chem.SDWriter('mol3.sdf')
    #writer.write(AllChem.EmbedMolecule(m[0], AllChem.ETKDGv2()))
    #writer.close() 
'''writer = Chem.SDWriter('mol3.sdf')
writer.write(x[0])
writer.close()'''
m[1]

IndexError: list index out of range

In [32]:
for m in x:
    print(GetAtoms(m[0]))

NameError: name 'GetAtoms' is not defined

In [31]:
import os
import pickle
from rdkit import RDConfig
m = Chem.MolFromMolFile(os.path.join(RDConfig.RDCodeDir, 'Chem', 'aligned_confs_puspita.mol'))
m.GetProp('_Name')

OSError: Bad input file D:\bld\rdkit_1634805116549\_h_env\Lib\site-packages\rdkit\Chem\aligned_confs_puspita.mol